## CSE150A Milestone 3: GaussianHMM Weather Prediction Model

In [ ]:
#!pip install hmmlearn #if necessary
import numpy as np
import pandas as pd
from hmmlearn import hmm
from sklearn.preprocessing import StandardScaler, LabelEncoder

In [ ]:
weather = pd.read_csv('seattle-weather.csv')
weather.head()